## Python selenium 学习

#### 1、selenium 自动化配置浏览器

In [24]:
# -*- coding:utf-8 -*-
from selenium import webdriver
from fake_useragent import UserAgent

ua = UserAgent()
option = webdriver.ChromeOptions()

# 1、使用无头chrome报以下错误：selenium.common.exceptions.WebDriverException: Message: unknown error: DevToolsActivePort file doesn't exist
       ## 确保驱动版本对应浏览器的版本,浏览器添加了 --no-sandbox ,取消沙盒模式
option.add_argument('--no-sandbox')   # “–no-sandbox”参数是让Chrome在root权限下跑，必须时第一个设置的options参数


option.add_argument('--disable-dev-shm-usage')
# 2、配置为无头浏览器
option.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
# 3、配置浏览器窗口大小，浏览器分辨率
option.add_argument("window-size=1366,768") 
# 4、 C:\Users\qyn\Desktop 为设置的缓存文件路径（可以是相对路径）
option.add_argument(r'--disk-cache-dir=C:\Users\qyn\Desktop')   
      ## window 读取文件可以用\，但是在字符串中\是被当作转义字符来使用，所以’d:\a.txt’会被转义成’d:\a.txt’这是正确路径，所以不会报错。
      ## Python 路径描述的三种方式：方式一:转义的方式：'d:\\a.txt'；方式二:显式声明字符串不用转义(原始字符串)：r'd:\a.txt'；
      ## 方式三:使用Linux的路径/：'d:/a.txt'（建议方式三：linux与Windows都有良好的支持）
# 5、禁止Python在使用chromedriver的headless模式下打印日志console信息
option.add_argument('log-level=3')   # chromedriver 在 headless 模式下运行会产生大量的日志信息,添加参数log-level
      ## 日志参数：INFO = 0, WARNING = 1, LOG_ERROR = 2, LOG_FATAL = 3，default is 0
# 6、配置浏览器的代理请求头——User-Agent
option.add_argument('user-agent="%s"' % ua.random) 
# 7、配置浏览器禁止弹窗以及默认下载路径
dirpath = r'C:\Users\qyn\Desktop'
prefs = {'profile.default_content_settings.popups': 0,     # 0 屏蔽弹窗，1开启弹窗
         "download.prompt_for_download": False,
         "download.directory_upgrade": True,
         "safebrowsing.enabled": True,
         'download.default_directory': dirpath}            # 设置下载路径
option.add_experimental_option('prefs', prefs)
# 8、谷歌文档提到需要加上这个属性来规避bug：仅适用于windows
option.add_argument('--disable-gpu')
# 9、不加载图片, 提升速度
option.add_argument('blink-settings=imagesEnabled=false')
# 10、隐藏滚动条, 应对一些特殊页面
option.add_argument('--hide-scrollbars') 
# chrome_options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"     #手动指定使用的浏览器位置
# 11、窗口最大化
option.add_argument('--start-maximized') 
# 12、设置字符编码
option.add_argument("lang=zh_CN.UTF-8")
# 13、隐身模式
option.add_argument("incognito")
# 14、阻止弹出框
option.add_argument('disable-infobars')
# 15、添加代理
proxy='http://127.0.0.1:8080'
option.add_argument('--proxy-server=' + proxy)

#  、启动浏览器控制
# print(help(webdriver.Chrome), flush=True)
browser = webdriver.Chrome(executable_path=r'D:\liing\chromedriver.exe',options=option)   # executable_path  驱动路径
#  、发出浏览器请求
browser.get('https://www.baidu.com') 
#  、无头浏览器请求截屏
browser.save_screenshot(r'D:\liing\baidu.png')
#  、关闭请求
# browser.close()
#  、退出浏览器控制
# browser.quit()


True

#### 2、selenium 自动化延时处理

**selenium 延时处理，针对的是网页加载缓慢的情况，主要有以下四种解决方法：**<br>

1）、显性等待WebDriverWait，配合该类的until()和until_not()方法，就能够根据判断条件而进行灵活地等待：<br>
　　WebDriverWait(driver, 超时时长, 调用频率, 忽略异常).until(可执行方法, 超时时返回的信息)<br>
　　WebDriverWait(driver, 超时时长, 调用频率, 忽略异常).until_not(可执行方法, 超时时返回的信息)<br>
  　WebDriverWait()会不断的根据你设定的条件去判断，直到超过你设置的等待时间，如果设置的条件满足，然后进行下一步操作，如果没有满足会报一个'selenium.common.exceptions.TimeoutException: Message: '错误。<br>
　　可以用selenium提供的 expected_conditions 模块中的各种条件，也可以用WebElement的 is_displayed() 、is_enabled()、is_selected() 方法，或者用自己封装的方法都可以

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait

browser.get('https://www.baidu.com') 
# 等待时长30秒，默认每0.5秒询问一次
WebDriverWait(browser, 30).until(lambda x: x.find_element_by_id("kw")).send_keys("sky")
# sel_count_info = WebDriverWait(driver, 30).until(lambda x: x.find_element_by_id("selCount")).text

2)、强制等待time.sleep()，不建议总用这种等待方式，太死板，严重影响程序执行速度

3)、隐性等待implicitly_wait(xx)，隐性等待对整个driver的周期都起作用，所以只要设置一次即可（全局）

In [22]:
# 隐性等待，最长等30秒 
# browser.implicitly_wait(30)  

4)、隐性显性并用

In [ ]:
from selenium.webdriver.support import expected_conditions as EC  
from selenium.webdriver.common.by import By  

browser.implicitly_wait(10)     # 隐性等待和显性等待可以同时用，但要注意：等待的最长时间取两者之中的大者  
browser.get('https://www.baidu.com')  
locator = (By.LINK_TEXT, '百度一下')  
try:  
    WebDriverWait(browser, 20, 0.5).until(EC.presence_of_element_located(locator))  
    print(browser.find_element_by_link_text('百度一下').get_attribute('href'))  
finally:  
    browser.close()  

# 隐性显性同时使用

#### 2、selenium 添加扩展

添加chrome扩展：<br>
option.add_extension('d:crxAdBlock_v2.17.crx') #自己下载的crx的路径<br>
browser = webdriver.Chrome(executable_path=r'D:\liing\chromedriver.exe',options=option)   # executable_path  驱动路径

#### selenium 常见异常

异常：  selenium.common.exceptions.ElementNotSelectableException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.InvalidElementStateException<br>
当尝试选择一个未能选中的元素时，异常会抛出<br>
异常：  selenium.common.exceptions.ElementNotVisibleException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.InvalidElementStateException<br>
虽然当一个元素呈现在DOM，但它是不可见的，因此不能够以进行交互，异常将抛出<br>
异常：  selenium.common.exceptions.ErrorInResponseException(response, msg)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
服务器端有错误时，异常将抛出（这个原因可能是因为Firefox插件或者远程server）<br>
异常：  selenium.common.exceptions.ImeActivationFailedException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
激活输入法失败时异常会抛出。<br>
异常：  selenium.common.exceptions.ImeNotAvailableException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
当输入法不支持的时候异常将抛出。<br>
异常：  selenium.common.exceptions.InvalidCookieDomainException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
试图在不同的domain而不是目前的URL中添加一个cookie时抛出异常<br>
异常：  selenium.common.exceptions.InvalidElementStateException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
无效的元素状态<br>
异常：  selenium.common.exceptions.InvalidSelectorException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.NoSuchElementException<br>
当选择器没有返回一个web元素时，异常抛出。<br>
异常：  selenium.common.exceptions.InvalidSwitchToTargetException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
当切换的窗口或者框架不存在的时候，异常将抛出。<br>
异常：  selenium.common.exceptions.MoveTargetOutOfBoundsException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
提供给ActionChainsmovable()方法的目标无效时，异常将抛出，例如：超出文件外<br>
异常：  selenium.common.exceptions.NoAlertPresentException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
切换到没有呈现的警示时抛出异常<br>
异常：  selenium.common.exceptions.NoSuchAttributeException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
当元素的属性不能被发现时异常抛出<br>
异常：  selenium.common.exceptions.NoSuchElementException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
元素不能被找到时异常抛出<br>
异常：  selenium.common.exceptions.NoSuchFrameException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.InvalidSwitchToTargetException<br>
需要切换的目标框架不存在时，异常抛出<br>
异常：  selenium.common.exceptions.NoSuchWindowException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.InvalidSwitchToTargetException<br>
需要切换的目标窗口不存在时，异常抛出<br>
异常：  selenium.common.exceptions.RemoteDriverServerException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
 
异常：  selenium.common.exceptions.StaleElementReferenceException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
一个参考的元素现在是“过时”时抛出异常。“过时”是指这个元素不再出现在页面的Dom中。<br>
异常：  selenium.common.exceptions.TimeoutException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
一条命令在足够的时间内没有完成则会抛出异常<br>
异常：  selenium.common.exceptions.UnableToSetCookieException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
当一个驱动程序无法设置cookie时抛出异常。<br>
异常：  selenium.common.exceptions.UnexpectedAlertPresentException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
当一个意外的警告出现时将抛出异常。<br>
异常：  selenium.common.exceptions.UnexpectedTagNameException(msg=None, screen=None, stacktrace=None)<br>
依据：  selenium.common.exceptions.WebDriverException<br>
辅助类没有获取到期待的web元素时，会抛出异常<br>
异常：  selenium.common.exceptions.WebDriverException(msg=None, screen=None, stacktrace=None)<br>
依据：  exceptions.Exception<br>
    WebDriver基础的异常类<br>